---
**Execute this tutorial in Binder** [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/MolSSI/zenopy/5a28d6babe6d197bfd3725f0b207065789fc0a9a?urlpath=lab%2Ftree%2Fdocs%2Fnotebooks%2Fquick-start.ipynb)

---

This guide demonstrates how you can use ``zenopy`` to search through Zenodo's public records and
retrieve individual records from the search results. 
For more tutorials, see the [documentation](https://molssi.github.io/zenopy).

### **Create a config file**

---
**Note**

Following this section is not required for performing search operations 
through Zenodo's public records. However, Zenodo 
[recommends](https://developers.zenodo.org/#authentication) 
all API requests be authenticated and sent over HTTPS.

---

When you want to use ``zenopy`` for the first time, you need to give it access to 
your Zenodo (or Zenodo Sandbox) account. The first step is to store your authentication
token(s) in a config file. You can create a new text file (say, **.zenodorc**) in
the current directory and store the token in it. The following code cell uses the
``%%writefile`` magic function to create the **.zenodorc** config file in the
current directory.

In [ ]:
%%writefile ./.zenodorc
[ZENODO]
token = 

[SANDBOX]
token = 

---
**Note**

If you do not know how to create an authentication token, refer to the [documentation](https://molssi.github.io/zenopy/howtos/client/cli_token.html).

---

Copy the token from your Zenodo account and paste it in the **[Zenodo]** section. 
The token stored in the config file will be used by Zenodo servers to authenticate your API requests.

### **Initialize the client**

The next step is to import the ``zenopy`` package

In [ ]:
import zenopy

The recommended way is to call the ``zenopy``'s client constructor and 
tell it where the config file is

In [ ]:
cli = zenopy.Zenodo(config_file_path="./.zenodorc", use_sandbox=False)

However, because we plan to use ``zenopy`` only for searching through Zenodo's 
public (and published) records in this tutorial, we can create the client object
without an authentication token (an empty string).

In [ ]:
cli = zenopy.Zenodo(token="", use_sandbox=False)

---
**Note**

Do not forget to set the ``use_sandbox`` argument to ``False``.
Otherwise, you will be searching through Zenodo Sandbox's servers.

---

Great. Now, the ``zenopy`` client is connected to the Zenodo servers
and is ready to be used to search through the records.

### **Search Through Zenodo’s public records**

In order to work with the records, we need to create an instance 
of the ``_Records`` class

In [ ]:
rec_obj = cli.init_records()
rec_obj

Now, we can call the **rec_obj**'s ``list_records()`` to search 
through Zenodo's public records.

In [ ]:
rec_list = rec_obj.list_records()
rec_list

By default, ``list_records()`` returns a list of 10 records.
You can change the number of records fetched by passing the ``size`` argument.

Before moving forward, let's create a utility function that prints the record indices
in the list and their corresponding titles. This will save us a little bit of 
time later on when we want to inspect the results of other search queries.

In [ ]:
def print_record_titles(record_list: list[zenopy.record.Record] = None) -> None:
    if record_list is None:
        raise TypeError("The 'record_list' argument cannot be None.")
    for idx, rec_idx in enumerate(record_list):
        print(idx, rec_idx.title)

Now, let's try our utility function on the record list we just obtained

In [ ]:
print_record_titles(record_list=rec_list)

As you can see, a general search will give us a random set of records that might not be
interesting to our research. Let's narrow our search down and focus on the public 
records that are available in the 
[MolSSI Zenodo Community](https://zenodo.org/communities/molssi/?page=1&size=20).

In [ ]:
molssi_rec_list = rec_obj.list_records(communities="molssi")

This list is a sample of scientific data that have become available by
the members of the computational molecular sciences community. Let's inspect
the list using our ``print_record_titles()`` utility function.

In [ ]:
print_record_titles(molssi_rec_list)

### **Retrieving individual records**

Sometimes, we may be interested in retrieving individual 
records from the results of our previous search and inspect their
metadata more closely. To do so, we need the unique record ID
that can be extracted from records' ``_id`` attribute

In [ ]:
des5m_id = molssi_rec_list[4]._id

Now, we can fetch the record object corresponding to this ID
using the ``retrieve_record()`` as shown below

In [ ]:
rec_des5m = rec_obj.retrieve_record(id_=des5m_id)

We can now access all metadata via record's ``data`` attribute

In [ ]:
rec_des5m.data

As the resulting information is in JSON format, accessing different
fields in the metadata is a convenient task. For example, let's print
the list of the authors behind the **DES5M** dataset

In [ ]:
rec_des5m.metadata["creators"]